# Propagação de epidemias em redes multicamada

Alunos:<br>
Caio Marcos Chaves Viana - 11208217<br>
Maria Teresa Raetano Cardoso - 10748604<br>
Paulo Matana da Rocha - 10892676

## Introdução

Esse trabalho visa comparar a propagaçã de epidemias em redes complexas, com uma ou mais camadas. Para isso desenvolvemos um pequeno fremework que realiza a conversão de redes geradas pela biblioteca Networkx em redes multicamada presentes na bilioteca pymnet. Também criamos funções capazes de simular os modelos de epidemia SIS, SIR e SIRS em redes com multiplas camadas.

## Modelagem em multiplas camadas

As simulações usadas no projeto seguem a ideia de uma maquina de estado que simula uma cadeia de Markov. O modelo realiza uma simulação com tempo discreto e atualização sincrona dos individuos, ou seja, todos os individuos são atualizado antes de realizar a atualização do tempo.
Em redes de uma camada temos que a simulação pode ser explicada pela imagem a seguir:

<img src="models_SL.png" width="400"/>

Nas redes multicamadas os vertices de mesmo indice em diferentes camadas indicam o mesmo individuo em redes diferentes. Dois o mesmo individuo pode possuim uma relação que conecta ele a ele mesmo em diferentes camadas, essa ligação é chamada de **inter-aresta** e pode possuim um valor que represente sua força. Pra modelar a propagação de epidemias em redes com mais de uma camada levando em consideração esses conceitos criamos um sistema onde o vértice pode infectar a si mesmo em outras camadas por meio de uma **inter-aresta**, além disso consideramos que a epidemia trata-se de um mesmo agente em todas as camadas, dessa forma quando um individuo se recupera em uma camada ele se recupera em todas. Podemos entender melhor o sistema olhando o diagrama a seguir, que representa o modelo SIR em uma rede de 3 camadas:

<img src="SIR_ML.png" width="500"/>

No esquema vemos a progressão de um unico individuo nas camadas **A**, **B** e **C**. Podemos ver que nesse modelo existe um valor $\lambda$ que representa a força da ligação intercamada desse individuo. Para os modelos SIS e SIRS temos criamos modelos analogos onde um infectado pode infectar a si mesmo em outras camadas.